In [ ]:
!pip install tensorflow
!pip install onnx
!pip install tf2onnx


In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]) # 2048 MB = 2GB
    except RuntimeError as e:
        print(e)

In [20]:
import tensorflow as tf
import tf2onnx

model = tf.keras.models.load_model('/content/drive/MyDrive/Model_vgg16.keras')
onnx_model_path = '/content/drive/MyDrive/Model_vgg16.onnx'

spec = (tf.TensorSpec((None, 256, 256, 3), tf.float32, name="input"),)
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=onnx_model_path)

print(f"Model saved to {onnx_model_path}")



Model saved to /content/drive/MyDrive/Model_vgg16.onnx


In [1]:
onnx_model_path = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\Model_resnet50v2.onnx"

In [2]:
import onnxruntime as ort

session = ort.InferenceSession(onnx_model_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
import time

class_labels = ['chakri', 'gowthami', 'kavya', 'kumar', 'sai', 'uday', 'vijay']

image_folder = r"C:\Users\Lenovo\Desktop\Face_identification\Main_data\10"

batch_start_time = time.time()

for img_name in os.listdir(image_folder):
    img_path = os.path.join(image_folder, img_name)

    img = image.load_img(img_path, target_size=(256, 256))
    input_tensor = image.img_to_array(img)
    input_tensor = np.expand_dims(input_tensor, axis=0).astype(np.float32)

    start_time = time.time()
    output = session.run([output_name], {input_name: input_tensor})[0]
    end_time = time.time()

    prediction_time = end_time - start_time

    probabilities = np.exp(output) / np.sum(np.exp(output), axis=1, keepdims=True)
    predicted_class_index = np.argmax(probabilities)

    print(f"Image: {img_name}")
    print(f"Predicted class: {class_labels[predicted_class_index]}")
    print(f"Time taken for prediction: {prediction_time:.6f} seconds")

batch_end_time = time.time()
total_batch_time = batch_end_time - batch_start_time

print(f"\nTotal time for processing all images: {total_batch_time:.6f} seconds")


Image: chakri (1).jpg
Predicted class: chakri
Time taken for prediction: 0.200144 seconds
Image: chakri (2).jpg
Predicted class: chakri
Time taken for prediction: 0.063412 seconds
Image: gowthami (1).jpg
Predicted class: gowthami
Time taken for prediction: 0.075404 seconds
Image: gowthami (2).jpg
Predicted class: gowthami
Time taken for prediction: 0.069070 seconds
Image: kumar (1).jpg
Predicted class: kumar
Time taken for prediction: 0.075992 seconds
Image: kumar (2).jpg
Predicted class: kumar
Time taken for prediction: 0.065212 seconds
Image: Sri (1).jpg
Predicted class: kavya
Time taken for prediction: 0.058474 seconds
Image: Sri (2).jpg
Predicted class: kavya
Time taken for prediction: 0.050971 seconds
Image: vijay (1).jpg
Predicted class: vijay
Time taken for prediction: 0.058766 seconds
Image: vijay (2).jpg
Predicted class: vijay
Time taken for prediction: 0.059441 seconds

Total time for processing all images: 0.892653 seconds
